<a href="https://colab.research.google.com/github/zxc-aix/favouriteTeacher/blob/main/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%BC%D0%B0%D1%81%D0%BE%D0%BA_%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%B4%D0%BB%D1%8F_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87_%D1%81%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Используем SAM для решения задачи

## Окружение Setup

не забываем установить использование GPU

In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.1.0+cu121
Torchvision version: 0.16.0+cu121
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-78xr2po1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-78xr2po1
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36587 sha256=75e182c6a17cfa87ab4e251841ccca6e60f33719d3e0ef0f546556807cbda8ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-u75gj8sl/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2024-01-15 06:18:16--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com

## маска

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

## Работаем с изображением

In [ ]:
# Сборка Google Drive directory and files
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)



Mounted at /content/gdrive/


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/SegAny
! pwd

%cd /mydrive/SegAny
! ls
! pwd

automatic_mask_generator_example.ipynb	predictor_example.ipynb  sam_vit_h_4b8939.pth.2
images					sam_vit_h_4b8939.pth
onnx_model_example.ipynb		sam_vit_h_4b8939.pth.1
/content
/content/gdrive/My Drive/SegAny
automatic_mask_generator_example.ipynb	predictor_example.ipynb  sam_vit_h_4b8939.pth.2
images					sam_vit_h_4b8939.pth
onnx_model_example.ipynb		sam_vit_h_4b8939.pth.1
/content/gdrive/My Drive/SegAny


In [ ]:
image = cv2.imread('images/micro_1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

## Генерация маски

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
masks = mask_generator.generate(image)

Генерация возвращает список масок, где каждая маска - словарь, содержащий различные данные о маске.

Ключи:

segmentation: маска

area: площадь маски в пикселях.

bbox: граничная рамка маски в формате XYWH.

predicted_iou: собственный прогноз модели относительно качества маски.

point_coords: выбранная входная точка, которая сгенерировала эту маску

stability_score: дополнительный показатель качества маски.

crop_box: обрезка изображения, используемого для создания этой маски в формате XYWH.

In [ ]:
print(len(masks))
print(masks[0].keys())

Все маски сгенерированные на изображении

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()

## Автоматическое создание масок

настраиваемые параметры для автоматической генерации:

In [ ]:
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # нужна open-cv
)

In [ ]:
masks2 = mask_generator_2.generate(image)

In [ ]:
len(masks2)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks2)
plt.axis('off')
plt.show()